In [ ]:
! pip install flair

In [ ]:
import numpy as np
import torch

In [ ]:
import pandas as pd

data = pd.read_csv("papers.csv")

In [ ]:
abstracts = data.cleaned_abstract_sentences.tolist()

In [ ]:
for a, t in zip(abstracts, data.cleaned_title):
    a.append(t)

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings, WordEmbeddings, TransformerWordEmbeddings
from flair.data import Sentence

# document embedding is a mean over GloVe word embeddings
pooled_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')], pooling='mean')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# document embedding is a pre-trained transformer
# transformer_embeddings = TransformerDocumentEmbeddings('bert-base-uncased')

# we want to do pooling for our baseline
pooled_embeddings = DocumentPoolEmbeddings([TransformerWordEmbeddings('bert-base-uncased')], pooling='mean')

In [ ]:
def embed_docs(abstracts, mode="bert"):
    sentencized = [Sentence(s) for a in abstracts for s in a]
    if mode == "bert":
        transformer_embeddings.embed(sentencized)
        return [s.embedding for s in sentencized]
        # return torch.mean(torch.stack([s.embedding for s in sentencized]), dim=0)
    elif mode == "glove-mean":
        pooled_embeddings.embed(sentencized)
        return [s.embedding for s in sentencized]
        # return torch.mean(torch.stack([s.embedding for s in sentencized]), dim=0)


In [ ]:
# all_abstract_bert_embeddings = [embed_doc(a) for a in abstracts]

In [ ]:
def create_alignment(df):
    """
    Map the sentence position indexes of the abstract sentences for a whole
    papers dataframe. Used later to map the separately calculated sentence 
    embeddings (due to performance gains in SBERT batch embedding) to 
    the correct paper.
    
    :param df: The papers dataframe.
    
    :return alignment: The sentence-to-position alignment dictionary.
    """
    alignment = {}
    current_start = 0
    current_end = 0
    for index, row in df.iterrows():
        abstract = row.abstracts_no_empties_no_toolong
        current_end += len(abstract)
        alignment[index] = {"start": current_start, "end": current_end}
        current_start = current_end
    return alignment

In [ ]:
alignment = create_alignment(data)

In [ ]:
all_sentence_embeddings_bert = embed_docs(abstracts)

In [ ]:
all_sentence_embeddings_glove = embed_docs(abstracts, "glove-mean")